## thoughts
* 3dGS render looks better than 2dGS
* it seems that good reconstructions cun be achieved also with 3dGS
* for the flies, the 2dGS gives good results, need to retest the 3d - in time
* the reconstruction in 2D is based on the depth map alone
* why not treat the 3dGS as havving a direction in the smallest scale? 
    * problematic with spheres, but if initiating a pita one and adding normal regularization, why would it become a sphere?
* everything is pixelwise - there is an article with patches
* examining other views in the context of reconstruction (epipolar, saw something, not sure were, need to find)
* something with the lightfield? the rendering of 3d is really good
* send a ray and sample (maybe saw something, need to find)? + multiview - get the most similiar feature on the ray 
* combine stereo reconstruction in the optimization process

* structured? a graph like structure that enables gaussians to share data and define normals as a neigberhood. 
    * each gaussian is a vertex? 
    * an interpulated color as in plenoxels? 
    * a new gaussian that represents the interpulation and normal? could be 2D
    * will help transperacy? 


* sparse view
    * use rendered view for reconstruction
    * use 3d/2d/3d with normal to render images
    * use the new images to get a more detailed depth map and regularize the normals
    * find "good" renders in image. doesnt have to be the whole image, can be patch - mesure quality
    * we can understand what points are ocluded by Z buffer - maybe add there more points


# 3D Gaussian splatting
## Elliptical Gaussian (from EWA article) - Math explained


we define an elliptical gaussian: 

Equation 1: $G_\mathbf{V}(\mathbf{x} - \mathbf{p}) = \frac{1}{2 \pi |\mathbf{V}|} \frac{1}{2} e^{-\frac{1}{2} (\mathbf{x} - \mathbf{p})^T \mathbf{V}^{-1} (\mathbf{x} - \mathbf{p})}.$

* $V$ is the covariance matrix
* $\mathbf{x}$ - a 3d coordinate (in gaussian axes)
* $\mathbf{p}$ - the mean of the gaussian 
### Affine projection of the gauusian
We define an arbitrary affine mapping from object space $ \mathbf{u} = \Phi(\mathbf{x}) $, where $ \Phi(\mathbf{x}) = \mathbf{Mx + c} $, 

and we substitute $ \mathbf{x} = \Phi^{-1}(\mathbf{u}) $. Since $ \Phi^{-1}(\mathbf{u}) = \mathbf{M}^{-1}(\mathbf{u} - \mathbf{c}) $, we get:

$$
G_\mathbf{V}(\Phi^{-1}(\mathbf{u}) - \mathbf{p}) = 
\frac{1}{|M^{-1}|} \cdot \frac{1}{(2 \pi)^{3/2} |\mathbf{V}|^{1/2}} e^{-\frac{1}{2} (\mathbf{M}^{-1}(\mathbf{u} - \mathbf{c}) - \mathbf{p})^T \mathbf{V}^{-1} (\mathbf{M}^{-1}(\mathbf{u} - \mathbf{c}) - \mathbf{p})} = 
$$

$$
\frac{1}{|M^{-1}|} \cdot \frac{1}{(2 \pi)^{3/2} |\mathbf{V}|^{1/2}} e^{-\frac{1}{2} (\mathbf{u} - \mathbf{c} - \mathbf{Mp})^T \mathbf{M}^{-1} \mathbf{V}^{-1} (\mathbf{M}^{-1})^T (\mathbf{u} - \mathbf{c} - \mathbf{Mp})} = 
$$

$$
\frac{1}{|M^{-1}|} \cdot \frac{1}{(2 \pi)^{3/2} |\mathbf{V}|^{1/2}} e^{-\frac{1}{2} (\mathbf{u} - \Phi(\mathbf{p}))^T (\mathbf{MVM}^T)^{-1} (\mathbf{u} - \Phi(\mathbf{p}))}.
$$

from this, to ease the writing we define equation 2: 

Equation 2: $G \mathbf{V}(\Phi^{-1}(u) - \mathbf{p}) = \frac{1}{|\mathbf{M}^{-1}|} G_{ \mathbf{M} \mathbf{V} \mathbf{M}^T} (u - \Phi(\mathbf{p})).$

#### The covariance matrix
$V$, the covariance matrix is a symetrix matrix that can be ragarded as a transformation. it scales and shears the data (thus defining the orientation and ratio between the gaussian principal axes). 
As mentioned above, the gaussian is transformed using an affine mapping $\Phi(X)$. Usually we use this kind of mapping to transform a vector or a 3D point to a new FoR (change basis). Because the covariance matrix is a transformation, in order to change its FoR (basis), we will perform $\mathbf{V_{1}} = MVM^{T}$. We first multiply $VM^{T}$  - transform (whatever xyz we multiply) from camera to body ($M^{T}$) and then multiply by $V$ which is in object coordinates - now we have coordinates in object basis after shear and scale. we then multiply by $M$ again to transform back to camera coordinates. 


### Projection


![alt text](ray_ewa.png)
#### Ray space

We converted to camera space, now we want to convert to ray space. The coordinates in ray space are defined such that $x_0, x_1$ are the coordinates on the image plane and the devision by $u_2$ adds prespective. $u_3$ is then defined as the euclidean distance from the point to the origin (camera). 


$$ \text{ray space to camera: }
\begin{pmatrix}
x_0 \\
x_1 \\
x_2 \\

\end{pmatrix} = m(u) = \begin{pmatrix}
u_0 / u_2 \\
u_1/u_2 \\
||(u_0, u_1, u_2)||^{T} \\
\end{pmatrix}

\text{,camera to ray space: }
\begin{pmatrix}
u_0 \\
u_1 \\
u_2 \\

\end{pmatrix} = m^{-1}(x) = \begin{pmatrix}
x_0/||(x_0,x_1,1)^T|| \cdot x_2 \\
x_1/||(x_0,x_1,1)^T|| \cdot x_2 \\
1/||(x_0,x_1,1)^T|| \cdot x_2 \\

\end{pmatrix}


$$
  
Ray space is defined with 3 coordinates $(x_0,x_1,x_2)$ where $x_0$ and $x_1$ represents the location of the pixel and $x_3$ is the euclidean distance from the camera origin to $(x_1,x_2). This transformation is not affine due to the devision by $u_2$, parallel lines are not preserved and the shape of the gaussian will be distorted. (in addition, the dimention is problematic).
In order to solve this a local affine approximation $\mathbf{m_{u_k}}$ of the projection transformation is defined. The approximation is the first to terms of the Tayor expansion of $\mathbf{m}$ at point $\mathbf{u}_{k}$:
$$\mathbf{m_{u_k}\mathbf(u)} = \mathbf x_k+\mathbf J_{\mathbf{u}_k}\cdot ( \mathbf u-\mathbf u_k )$$
where $\mathbf x_k=\mathbf m(\mathbf u_k)$ is the mean of the gaussian in ray space (can be transformed using the mapping m, the projection matrix) and the Jacobian $ J_{u_k} = \frac{\partial m}{\partial u}(u_k) $

### The gaussian mapping
#### Object to camera coordinates (as detailed in Affine projection of the gauusian)

The reconstruction kernels are initially given in object space, which has coordinates $t =(t_0,t_1,t_2)^T$ . 
* Gaussian reconstruction kernels in object space: $r"_k (t)=G_{V"_k} (t − t_k)$, where $t_k$ are the voxel positions in object space
* Object coordinates are transformed to camera coordinates using an affine mapping $\mathbf u = \phi(t)$, called viewing transformation. It is defined by a matrix $\mathbf W$ and a translation vector $\mathbf d$ as $\phi(t) = Wt + d$. We transform the reconstruction kernels $G_{V"k} (t − t_k)$ to camera space by substituting $t = \phi ^{−1}(u)$ and using Equation 2 : 

$G \mathbf{V"_{k}}(\Phi^{-1}(u) - t_k) = \frac{1}{|\mathbf{W}^{-1}|} G_{V'_k } (\mathbf u - \mathbf u_k) = r'_k(u\mathbf).$

where $u_k = \phi(\mathbf t_k)$ is the center of the Gaussian in camera coordinates and $r′_k(\mathbf u)$ denotes the reconstruction kernel in camera space. According to equation 2, the covariance matrix in camera coordinates $V′_k$ is given by $V′_k = WV"_kW^T$ (exactly as detailed in Affine projection of the gauusian).

#### Camera to ray space
Reminder: the local affine transformation - $$\mathbf{m_{u_k}\mathbf(u)} = \mathbf x_k+\mathbf J_{\mathbf{u}_k}\cdot ( \mathbf u-\mathbf u_k )$$
$\mathbf{m_{u_k}\mathbf(u)}$ transforms from camera to ray, hence $x_k=m(u_k)$ where $x_k$ is the center of the gaussian in ray space. 
since $\mathbf x = m(\mathbf u)$ we get that $ u = \mathbf m^{-1}(\mathbf(x))$ hence: 
$$ u = \mathbf m^{-1}(x) = \mathbf(x - x_k)\cdot \mathbf J^{-1}_{\mathbf u_k} + \mathbf u_k $$
$$ r_k(\mathbf x) = \frac{1}{|\mathbf W^{-1}|}\cdot G_{\mathbf V'_k}(\mathbf{(x - x_k)\cdot J^{-1}_{\mathbf u_k} + \mathbf u_k - \mathbf u_k}) = \\
\frac{1}{|\mathbf W^{-1}||\mathbf J^{-1}|}G_{\mathbf V'_k}(J^{-1}_{\mathbf u_k}\mathbf{(x - x_k)})\\$$

and the new covariance matrix is defined:  $Vk = JV'_{k}$ $J^T = JWV"_kW^TJ^T$.
where V"_k is the original covariance matrix





### Perspective Projection and Jacobian Derivation

In a pinhole camera model, the perspective projection of a 3D point \((X, Y, Z)\) into 2D image coordinates \((x, y)\) can be expressed as:

$$
\begin{bmatrix}
x \\
y \\
1
\end{bmatrix} = \frac{1}{Z} \cdot
\begin{bmatrix}
f_x \cdot X + c_x \cdot Z \\
f_y \cdot Y + c_y \cdot Z \\
Z
\end{bmatrix}
$$

Where:

* $(x, y)$  : are the projected 2D coordinates on the image plane.
* $(X, Y, Z)$  : are the coordinates in the 3D space. 
* $f_x , f_y$  : are the focal lengths in the x and y directions, respectively. 
* $c_x , c_y$  : are the coordinates of the principal point (optical center) in the image. 

### Derivation of the Jacobian

To derive the Jacobian, we compute the partial derivatives of the projected 2D coordinates \((x, y)\) with respect to the 3D coordinates \((X, Y, Z)\).

1. **From the perspective projection equations**:
   - For \(x\):
   $$
   x = \frac{f_x \cdot X + c_x \cdot Z}{Z}
   $$
   - For \(y\):
   $$
   y = \frac{f_y \cdot Y + c_y \cdot Z}{Z}
   $$

2. **Calculating the derivatives**:
   - The derivatives with respect to \(Z\) yield:
     - For \(x\):
     $$
     \frac{\partial x}{\partial X} = \frac{f_x}{Z}, \quad \frac{\partial x}{\partial Y} = 0, \quad \frac{\partial x}{\partial Z} = -\frac{f_x \cdot X}{Z^2}
     $$
     - For \(y\):
     $$
     \frac{\partial y}{\partial X} = 0, \quad \frac{\partial y}{\partial Y} = \frac{f_y}{Z}, \quad \frac{\partial y}{\partial Z} = -\frac{f_y \cdot Y}{Z^2}
     $$

3. **Forming the Jacobian**:
   Using the derivatives calculated above, the Jacobian matrix \(J\) becomes:
   $$
   J = \begin{bmatrix}
   \frac{f_x}{Z} & 0 & -\frac{f_x \cdot X}{Z^2} \\
   0 & \frac{f_y}{Z} & -\frac{f_y \cdot Y}{Z^2}
   \end{bmatrix}
   $$


### 2D Projection 
We calculate the conics coeffitient for each gaussian : $$Ax^2+Bxy+Cy^2 = 0$$, where $A = \frac{\sigma_y^2}{|V|}$, $B = \frac{\sigma_xy}{|V|}$ and  $C = \frac{\sigma_x^2}{|V|}$
we then multiply by the distace between the gaussiam and the pixel. this is the same as: 
$-\frac{1}{2} (\mathbf{x} - \mathbf{p})^T \mathbf{V}^{-1} (\mathbf{x} - \mathbf{p})$ - the exponent of the gaussian

wIncorporating the scaled conic, which represents the 2D Gaussian as an ellipse in the Gaussian equation, we can express the effect of every Gaussian on the pixel's color as follows:

$$
G_{\mathbf{V}}(\mathbf{x}) = \frac{1}{(2\pi)^{3/2} |\mathbf{V}|^{1/2}} e^{-\frac{1}{2} ( Ax^2+Bxy+Cy^2)}
$$

This equation captures how each Gaussian contributes to the pixel's color based on its distance from the pixel and the shape defined by the covariance matrix.

## tiles
To render the image we devide it to tiles of $ 16 X 16 $, for each tile we sort the gaussians according to the distance from the camera. we then sample all gaussians effect per pixel in the tile (using the power equation to calculate the opacity). We sum the gaussians color values and multiply by the opacity (doing alpha blending, each time we update the opacity according to the already calculated gaussians $color = RGB * \alpha *\alpha_{blend})$ were for eacg gaussian we add we update $\alpha_{blend} =  \alpha_{blend}*(1-\alpha) $ (when we reach 0, its opaque). to later do the alpha blend we pick for each tile only the gaussians that intersect with the tile. since gaussians are not bounded, we bound them with 3 $\sigma_x$ and 3 $\sigma_y$ bounding box in each axis. 



# 2D Gaussian splatting

## 2DGS - Math explained
### Modeling: 
The 2D Gaussian is defined in terms of its local coordinate system, where the $ X$ and $ Y$ axes are scaled according to the Gaussian's shape. The Gaussian is represented in the **world coordinates**, with the **axes** of the Gaussian in world space defined by the tangential vectors $ \mathbf{t_u} $ and $ \mathbf{t_v} $, and its **scaling factors** by $ s_u $ and $ s_v $. These tangential vectors define the directions of the local coordinate axes in the tangent plane (the object FoR).

The **object plane**, which is the tangent plane to the Gaussian in world space, can be described using the plane equation:
$$
P(u, v) = p_k + s_u \mathbf{t_u} u + s_v \mathbf{t_v} v
$$
where $ p_k $ is the center of the Gaussian in world coordinates, and $ u $ and $ v $ represent local coordinates on the tangent plane. This equation expresses the position of any point in the tangent plane in terms of the world coordinates, as modified by the scaling along the tangential axes $ \mathbf{t_u} $ and $ \mathbf{t_v} $.

To transform the local coordinates in the tangent plane (object frame of reference) to world coordinates, we define the following transformation matrix $ H $, which encodes the **scaling**, **rotation**, and **translation** from the object space to the world space:
$$
H = \begin{pmatrix}
s_u \mathbf{t_u} & s_v \mathbf{t_v} & 0 & p_k \\
0 & 0 & 0 & 1
\end{pmatrix}
= \begin{pmatrix}
R & p_k \\
0 & 1
\end{pmatrix}
$$
where:
- $ \mathbf{t_u} = \begin{pmatrix} t_{u_x} \\ t_{u_y} \\ t_{u_z} \end{pmatrix} $ and $ \mathbf{t_v} = \begin{pmatrix} t_{v_x} \\ t_{v_y} \\ t_{v_z} \end{pmatrix} $ are the 3D tangential vectors (defining the axes of the tangent plane in world coordinates),
- $ p_k = \begin{pmatrix} p_{k_x} \\ p_{k_y} \\ p_{k_z} \end{pmatrix} $ is the 3D position of the Gaussian center in world coordinates, and
- $ R $ is the 3x3 rotation matrix that describes the orientation of the tangent plane in world space.

The matrix $ H $ can be interpreted in two parts:
- The first part, $ \begin{pmatrix} s_u \mathbf{t_u} & s_v \mathbf{t_v} & 0 \end{pmatrix} $, represents the scaling and rotation of the axes in the world coordinates. It scales the tangential vectors $ \mathbf{t_u} $ and $ \mathbf{t_v} $ by $ s_u $ and $ s_v $, respectively, and applies any rotational transformation.
- The second part, $ p_k $, represents the **translation** of the Gaussian's center in world coordinates, which shifts the origin of the local tangent plane to the desired world position.

The matrix can also be interpreted in terms of the rotation matrix $ R $ (which describes the orientation of the tangent plane in world space) and the translation vector $ p_k $. The transformation from local coordinates $ (u, v) $ to world coordinates $ (x, y, z) $ is thus achieved through the multiplication of $ H $ by the local coordinate vector:
$$
\begin{pmatrix}
x \\
y \\
z \\
1
\end{pmatrix}
= H \begin{pmatrix}
u \\
v \\
1 \\
1
\end{pmatrix}
$$

This transformation allows us to map a point in the tangent plane (object space) to the corresponding point in the world space, accounting for both the Gaussian's shape (through the scaling $ s_u, s_v $) and its orientation (through the rotation $ R $).

In summary, the matrix $ H $ serves as a **homogeneous transformation** that encapsulates both the geometric properties (scaling and rotation) and the positioning (translation) of the 2D Gaussian in world space.

for every u,v on the tangent plane we can calculate the gaussian power: 
$$ G(u) = \exp\left( -\frac{u^2 + v^2}{2} \right) \tag{6} $$

* $ \mathbf{t_u} $ , $ \mathbf{t_v} , $ $\mathbf{s_u} $ and $\mathbf{s_v}$ are learnable parameters.
* each gaussian is defined by the opacity $\alpha$ and a view dependent color.

### Splatting


The goal of this process is to find the intersection between a ray, originating from a pixel in the image, and the tangent plane of a 2D Gaussian, and then evaluate the power of the Gaussian at the intersection point. Below are the detailed steps.

Step 1: Defining the Image Ray 

We begin by defining the ray from a pixel in the image using two orthogonal planes. The pixel location $(x, y) $ in the screen space can be used to define two planes:


* The x-plane (yz): a plane defined by a normal vector $ \mathbf{n}_x = (-1, 0, 0) $ and an offset $ x $. The 4D homogeneous form is $ h_x = (-1, 0, 0, x) $.
* The y-plane (xz): a plane defined by a normal vector $ \mathbf{n}_y = (0, -1, 0) $ and an offset $ y $. The 4D homogeneous form is $ h_y = (0, -1, 0, y) $.


These planes intersect at a ray in 3D space that is represented as the line of intersection between the two planes.

Step 2: Transforming the Planes to the Tangent Plane

Next, we transform the planes from the image space into the local coordinates of the 2D Gaussian primitive (the tangent plane). Using the transformation matrix $ M = (WH)^{-1} $, we apply the inverse transpose to the planes (W is the prespective projection matrix (transformation from camera to screen space), H is a transformation from tangant plane to camera space):

multiplying W (camera to screen) by H (tangent plane to camera) will transform a coordinate from tangent plane to camera to screen. Transposing it will map from screen space to tangent plane 

$$
h_u = (WH)^\top h_x \\
h_v = (WH)^\top h_y
$$
* those are the planes rotated from image space to tangent space


- We can imagine a ray from the camera center through a pixel, The pixel is described in screen space (here we use NDC) which means we transform it twice. 
1. from NDC to camera coordinate
2. from camera coordinates to gaussian (objec) coordinates. 
* The ray stays the same, it is still begins in the camera center and pass through the pixel. After all transformations we have the ray as seen from the gaussian axes. 

Step 3: Solving for the Intersection

The next step is to solve for the intersection point $(u, v) $ of the ray with the transformed tangent plane. We do this by solving the following system of equations:

$$
h_u \cdot (u, v, 1, 1)^\top = 0 \\
h_v \cdot (u, v, 1, 1)^\top = 0
$$

Expanding these equations gives us:


$$
    h_1^u u + h_2^u v + h_3^u + h_4^u = 0 \\
    h_1^v u + h_2^v v + h_3^v + h_4^v = 0
$$


The solution for $ u $ and $ v $ is obtained by solving this system of equations. This leads to the following expressions for the coordinates $ u(x) $ and $ v(x) $:

$$
u = \frac{h_2^u h_4^v - h_4^u h_2^v}{h_1^u h_2^v - h_2^u h_1^v} \\
v = \frac{h_4^u h_1^v - h_1^u h_4^v}{h_1^u h_2^v - h_2^u h_1^v}
$$

Where $ h_i^u $ and $ h_i^v $ are the homogeneous parameters of the transformed planes.

Step 4: Evaluating the Gaussian at the Intersection

Once we have the coordinates $ (u, v) $, we can evaluate the 2D Gaussian function at the intersection point. The Gaussian function is typically of the form:

$$
G(u, v) = \exp\left( -\frac{u^2 + v^2}{2} \right)
$$

This represents the Gaussian value at the point $ (u, v) $ on the tangent plane.


Once the values of $ u(x) $ and $ v(x) $ are found, we can compute the depth of the intersection point using the following equation:
$$
x = (x_z, y_z, z, z)^\top = W P(u, v) = W H (u, v, 1, 1)^\top
$$
Here, $ W $ is the camera projection matrix, and $ H $ is the transformation matrix from the tangent plane to world coordinates. The last component of the resulting vector $ x = (xz, yz, z, z)^\top $ gives the depth $ z $ of the intersection point (the third element, need to make sure). 



#### Screen space
this transformation returns a 3d coordinate in screen space. 
, where rendering occures. the projection matrix transformes x,y,z to the screen space
* x axis spans from 0 to -1 (width)
* y axis spans from 0 to -1 (hight)
* Z_{NDC} represents the depth (need to check that)
from NDC we can map to pixels

#### Transforming planes
* A point in 3D homogeneous coordinates is represented as $ (x,y,z,w)
* a plane consist of both a normal and an offset - in 3D homogeneous coordinates it is represented as $(a,b,c,d)
where $(a,b,c) is the planes normal vector and $d$ is its offset. 

The plane equation : $ ax + by + cz + dw = 0 $
which means that that dot product of the plane parameters $ (a,b,c,d)$ and a point $(x,y,z,w)$ is zero for a point lying on the plane. 

** Transforming Point on a Plane **
we Define matrix $M$ - a transformation matrix. 
tansforming a point on the plain using $M$: $p'=M\cdot p$
after transforming it, the plane equation should still hold: $h'p' = 0$ were $h', p'$ are the transformed plane and point parameters.

Because $p' = M\cdot p$ we get that $h'(M\cdot p) = 0 $ and since $ h' \cdot(M\cdotp p)=M^T \cdot h' \cdot p $ we can write 
$$
M^T \cdot h' \cdot p = h\cdot p \\
M^T \cdot h' = h
$$

and from that we show that the plane $h'$ remains valid after transformation. 


#### Regularization terms

There are two regularization terms, the first Depth distortion and the second - Normal consistency.


### Normal consistency: 

$ L_n = \sum_i \omega_i (1 - \mathbf{n}_i^\top \mathbf{N})$

* n - the normal of the 2d gaussian, we calculate it by rotating the Z axis from world axes to camera axes. 

* N - the normal to the surface. we consider the actual surface at the median point of intersection, where the accumulated opacity reaches 0.5. 
The sampled point generate a 2d depth map (prespective projection). We then transform it to 3d surface in world coordinates. 
To do so we calculate a ray from the camera center to through the pixel. We transform it from the 2d image space to the camera space (3d) to the world space (3D). We then multiply each ray direction by the depth value at the corresponding pixel to "scale" the ray to the correct length, reaching the surface point. 
After calculating the surface we can calculate the gradients in X and Y direction, thus getting the normal to the tangent plane for every voxel of the surface. (finaly we normalize by this normal).
* $\omega_i$ - the blending weight of the i-th intersection

This term will try to take the dot product of both normals to 1, thus agjusting the splats to locally align with the surface
* the input depth map here is generated from the coordinates of the mean of the gaussians. Hence, if the gaussians are on the surface (achieved by the othe regularization term) then we will have a general estimation of the normals. 

### Depth distortion: 

$ L_d = \sum_{i,j} \omega_i \omega_j |z_i - z_j| $

* $\omega_{i/j}$ - the blending weight of the i-th intersection - close to one is opaque, close to zero is opaque, close to one is transperant. 
* $ z_i  $ - the intersection between a ray from the camera origin through the pixel to the splat. $ |z_i - z_j| $ is the distance between the intersection points. 


The term will make the splats close to each other and opaque - hence optimizing the surface of the object


# TSDF (used for mesh reconstruction)
* define a voxel grid, each voxel will contain TSDF value and weight
* for each view identify the voxels that are inside the frustum
* foar each of the voxels project to the depth map and sample the depth
* calculate the SDF between the voxel's depth and the measured depth from the image. 
* Retrieve the current TSDF value (TSDF_old) and weight (weight_old) stored in the voxel.
* calculate the new TSDF and Weight and update for the voxel:
    * $TSDF_{new weighted} = (TSDF_{old} * weight_{old} + TSDF_{new} * 1) / (weight_{old} + 1)$
    * $weight_{new total} = weight_{old} + 1$
* Store the TSDF_{new_weighted} and weight_{new_total} back into the voxel

### Reconstruct the surface
* Identify voxels where the absolute value of the TSDF is below a small threshold (e.g., 0.1). These voxels are considered to be part of the surface.
* Optionally, use algorithms like Marching Cubes to generate a continuous mesh representation of the surface from the TSDF volume.